# 1| Importação das bibliotecas e ajuste do notebook

In [1]:
from warnings import filterwarnings

filterwarnings('ignore')

In [2]:
import pandas as pd 
import numpy as np

import string
import unidecode

import re

import nltk 
from nltk.corpus import stopwords 

from sklearn.model_selection import train_test_split, GridSearchCV

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.svm import SVC

from keras.utils import to_categorical
from keras import backend as K

import tensorflow as tf

from sklearn.metrics import accuracy_score

# 2 | Leitura dos dados

In [3]:
# IMPORTANDO O DATASET 

df_raw = pd.read_csv("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\10. NLP - Análise de Toxidade\\train (2).csv")

In [4]:
# CRIANDO UM BACKUP

df = df_raw.copy()

# 3| Entendimento dos dados 

In [5]:
# DIMENSÕES DO DATASET

print("O dataset possui",df.shape[0],"linhas e",df.shape[1],"colunas")

O dataset possui 16800 linhas e 2 colunas


In [6]:
# PRÉ-VISUALIZAÇÃO DOS DADOS

df.head()

,text,label
0,"rt @user olha quem chegouuuuu, nossos queridin...",0
1,veio umas teorias muito loucas na minha cabeça...,1
2,@user @user 😂😂😂😂mais nao tinha falado ontem qu...,0
3,rt @user quer ser filha da puta logo comigo qu...,1
4,vai besta 😂😂😂😂 casquei com a ultima foto,1


In [7]:
# ANÁLISE DA PROPORÇÃO DE CADA UMA DAS CLASSES

proporcao = df['label'].value_counts().to_frame()

proporcao['%'] = (proporcao['count'] / df.shape[0]) * 100

proporcao['%'] = proporcao['%'].round()

proporcao

,count,%
label,,
0,9425,56.0
1,7375,44.0


# 4| Funções de limpeza de dados e Pipeline para limpar os dados automaticamente 

In [8]:
#### CRIANDO ALGUMAS FUNÇÕES PARA FACILITAR A LIMPEZA DE PARTE DOS DADOS

# retirar os links 
def remove_https(text):
    return re.sub(r'https\S+', '', text)

# remover emojis
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  
                               u"\U0001F300-\U0001F5FF"  
                               u"\U0001F680-\U0001F6FF"  
                               u"\U0001F1E0-\U0001F1FF"  
                               u"\U00002500-\U00002BEF"  
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Definindo as stopwords em ptbr
stop_words = set(stopwords.words('portuguese'))  
stop_words.add('rt')
stop_words.add('@user')
stop_words.add('\n')

# retirar stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

#### CRIANDO UMA FUNÇÃO DE PRE-PROCESSAMENTO PARA APLICAR TODAS DE UMA VEZ e RETORNAR O DADO LIMPO

def pipeline(text):
    # deixar todos os dados minúsculos
    text = text.lower()
    
    # retirar pontuações e sinais
    for i in string.punctuation:
        text = text.replace(i, "")
        
    # retirar os links 
    text = remove_https(text)
    
    # retirar emojis
    text = remove_emojis(text) # sem emojis
    
    # retirar acentos
    text = unidecode.unidecode(text)
        
    # retirar stopwords
    text = remove_stopwords(text)
        
    return text

In [9]:
# Criando uma coluna para armazenar os dados tratados/limpos
df['text_adjusted'] = df['text'].copy()

In [10]:
# Aplicando os tratamentos na coluna criada
df['text_adjusted'] = df['text_adjusted'].apply(pipeline)

In [11]:
# Visualização do resultado
df

,text,label,text_adjusted
0,"rt @user olha quem chegouuuuu, nossos queridin...",0,user olha chegouuuuu queridinhos vem direcao f...
1,veio umas teorias muito loucas na minha cabeça...,1,veio umas teorias loucas cabeca agora pqp to a...
2,@user @user 😂😂😂😂mais nao tinha falado ontem qu...,0,user user nao falado ontem nao ia patrocinado ...
3,rt @user quer ser filha da puta logo comigo qu...,1,user quer filha puta logo comigo 50x pior kkkk...
4,vai besta 😂😂😂😂 casquei com a ultima foto,1,vai besta casquei ultima foto
...,...,...,...
16795,performer da nação caralho,1,performer nacao caralho
16796,"vôlei feminino é foda né, pqp, só vem tóquio",1,volei feminino foda ne pqp so vem toquio
16797,"@user cara de pau, quem desrespeita a constitu...",1,user cara pau desrespeita constituicao federal...
16798,duas das grandes atletas do frescobol mundial....,0,duas grandes atletas frescobol mundial academi...


# 5| Separação de dados de treino e teste

In [12]:
# Criando um novo dataframe a ser usado no treinamento apenas com os dados tratados + label
df_modeling = df[['text_adjusted','label']]

In [13]:
# Separando em variável target e preditora
X = df['text_adjusted']
y = df['label']

In [14]:
# Criando os dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Instanciando o tokenizer
tokenizer = Tokenizer()

# Ajustando o tokenizer nos dados de treino
tokenizer.fit_on_texts(X_train)

# Aplicando o tokenizer aos dados de treino e teste
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Quantidade de palavras (será usada na camada de embedding)
qtd_palavras = len(tokenizer.word_index) + 1

In [16]:
## Padding -> to uniform the datas
max_length = max(len(seq) for seq in X_train)

# to test an outlier case (if one of the test dataset has longer length)
for x in X_test:
    if len(x) > max_length:
        print(f"an outlier detected: {x}")

X_train = pad_sequences(X_train, maxlen = max_length)
X_test = pad_sequences(X_test, maxlen = max_length)

In [17]:
# create hot_labels (idk whty tapi ini penting, kalo ga bakal error)
y_test = to_categorical(y_test, num_classes=2)
y_train = to_categorical(y_train, num_classes=2)

In [18]:
# another look on the number of tweet in test and training data

print(f"num test tweet: {y_test.shape[0]}")
print(f"num train tweet: {y_train.shape[0]}")

num test tweet: 3360
num train tweet: 13440


# 6| Criação do modelo 1 - LSTM

In [19]:

def recall(y_true, y_pred):
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    possible_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    predicted_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+tf.keras.backend.epsilon()))



In [20]:
# change dis if u want
output_dim = 200

# LSTM model architechture (CNN + LSTM)
model = Sequential([
    # embedding layer is like idk
    Embedding(qtd_palavras, output_dim,input_shape=(max_length,)),
    # lstm for xxx
    LSTM(64, dropout=0.3, recurrent_dropout=0.3),
    # dropout to prevent overfitting
    Dropout(0.5),
    # dense to connect the previous output with current layer
    Dense(128, activation="relu"),
    # dropout to prevent overfitting
    Dropout(0.5),
    # this is output layer, with 3 class (0, 1, 2)
    Dense(2, activation="softmax"),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1,precision, recall])

In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 55, 200)             │       4,104,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          67,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,180,818 (15.95 MB)

 Trainable params: 4,180,818 (15.95 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Train the model
model_history = model.fit(
    X_train,
    y_train,
    batch_size = 64,
    epochs=10,
    validation_data=(X_test, y_test)
)

Epoch 1/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 15s 59ms/step - accuracy: 0.5934 - f1: 0.5934 - loss: 0.6589 - precision: 0.5934 - recall: 0.5934 - val_accuracy: 0.7491 - val_f1: 0.7500 - val_loss: 0.5141 - val_precision: 0.7500 - val_recall: 0.7500
Epoch 2/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 12s 56ms/step - accuracy: 0.8113 - f1: 0.8113 - loss: 0.4240 - precision: 0.8113 - recall: 0.8113 - val_accuracy: 0.7446 - val_f1: 0.7447 - val_loss: 0.5336 - val_precision: 0.7447 - val_recall: 0.7447
Epoch 3/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 12s 56ms/step - accuracy: 0.8766 - f1: 0.8766 - loss: 0.2869 - precision: 0.8766 - recall: 0.8766 - val_accuracy: 0.7268 - val_f1: 0.7267 - val_loss: 0.6343 - val_precision: 0.7267 - val_recall: 0.7267
Epoch 4/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.9107 - f1: 0.9107 - loss: 0.2150 - precision: 0.9107 - recall: 0.9107 - val_accuracy: 0.7188 - val_f1: 0.7190 - val_loss: 0.7801 - val_precision: 0.7190 - val_recall: 0.7190
Epoch 5/10
210/210 ━━━━━━━━━━━━━

# 6.1| Predição Competição

In [23]:
# Download do conjunto de teste a ser predito na competição
df_teste = pd.read_csv("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\10. NLP - Análise de Toxidade\\test (4).csv")

In [24]:
# Aplicando o pipeline de tratamento de dados
df_teste['text'] = df_teste['text'].apply(pipeline)

In [25]:
# Transformando a coluna com os textos a serem classificados em 'array'
X_pred = df_teste['text']

# Tokenizando os textos
X_pred = tokenizer.texts_to_sequences(X_pred)

X_pred = pad_sequences(X_pred, maxlen = max_length)

In [26]:
# Predição
y_pred = model.predict(X_pred)

132/132 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [27]:
# Visualização das classificações feitas
y_pred

array([[9.9126071e-01, 8.7393140e-03],
       [9.9899036e-01, 1.0096114e-03],
       [5.8373843e-07, 9.9999940e-01],
       ...,
       [1.0000000e+00, 5.7883054e-09],
       [3.6541006e-01, 6.3458997e-01],
       [9.9125779e-01, 8.7422552e-03]], dtype=float32)

> Nota-se que a predição é a probabilidade do texto avaliado ser classificado para cada classe (coluna 0 = não tóxico e coluna 1 = tóxico)

In [28]:
# Limiar para decidir a classe
threshold = 0.7

# Convertendo as probabilidades para as classes finais usando um limiar
predictions = np.where(y_pred >= threshold, 1, 0)

In [29]:
# Convertendo as classificações em uma array
predicted_classes = np.argmax(predictions, axis=1)

In [30]:
# Salvando as classificações em uma nova coluna
df_teste['label'] = predicted_classes

In [31]:
# Excluindo a coluna de 'text' para manter o formato de submissão da competição no Kaggle
df_teste.drop('text', axis=1, inplace=True)

In [32]:
# Salvando o df_teste em um csv para ser submetido ao Kaggle
df_teste.to_csv("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\10. NLP - Análise de Toxidade\\Submissões\\submissao_lstm.csv", index=False)

> O modelo de LSTM obteve um resultado de xxx de acurácia nos dados da competição. Com isso, outros modelos serão também treinados e testados para tentar alcançar a melhor pontuação.

# 7 | Modelo SVC

In [33]:
# Divisão dos dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_modeling['text_adjusted'], df_modeling['label'], test_size=0.2, random_state=42)


# Vetorização do texto usando TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Treinamento do modelo SVM
model = SVC(kernel='linear')
model.fit(X_train_vect, y_train)

# Fazendo previsões
predictions = model.predict(X_test_vect)

# Avaliando o modelo
accuracy = accuracy_score(y_test, predictions)
print("Acurácia do modelo:", accuracy)

Acurácia do modelo: 0.743452380952381


In [ ]:
# Divisão dos dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df_modeling['text_adjusted'], df_modeling['label'], test_size=0.2, random_state=42)



# Instanciando o tokenizer
tokenizer = Tokenizer()

# Ajustando o tokenizer nos dados de treino
tokenizer.fit_on_texts(X_train)

# Aplicando o tokenizer aos dados de treino e teste
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Quantidade de palavras (será usada na camada de embedding)
qtd_palavras = len(tokenizer.word_index) + 1

## Padding -> to uniform the datas
max_length = max(len(seq) for seq in X_train)

# to test an outlier case (if one of the test dataset has longer length)
for x in X_test:
    if len(x) > max_length:
        print(f"an outlier detected: {x}")

X_train = pad_sequences(X_train, maxlen = max_length)
X_test = pad_sequences(X_test, maxlen = max_length)

# create hot_labels (idk whty tapi ini penting, kalo ga bakal error)
y_test = to_categorical(y_test, num_classes=2)
y_train = to_categorical(y_train, num_classes=2)

# Convertendo as classificações em uma array
y_test_array = np.argmax(y_test, axis=1)
y_train_array = np.argmax(y_train, axis=1)

# Treinamento do modelo SVM
model = SVC(kernel='linear')
model.fit(X_train, y_train_array)

# Fazendo previsões
predictions = model.predict(X_test)

# Avaliando o modelo
accuracy = accuracy_score(y_test_array, predictions)
print("Acurácia do modelo:", accuracy)

In [ ]:
# Convertendo as classificações em uma array
teste = np.argmax(y_test, axis=1)

In [ ]:
teste